In [2]:
%useLatestDescriptors
%use dataframe(1.0.0-Beta3), kandy

In [3]:
import com.fasterxml.jackson.databind.DeserializationFeature
import com.fasterxml.jackson.databind.ObjectMapper

val mapper = ObjectMapper()
mapper.configure(DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, true)

com.fasterxml.jackson.databind.ObjectMapper@79822590

In [7]:
import ds.types.raw.forum.ForumProfiles

val data = mapper.readValue<ForumProfiles>(File("C:/Users/Alex/Idea/ds/prj/data/FrequentFlyerForum-Profiles.json"), ForumProfiles::class.java)

In [8]:
println(data.profiles.size)

66592


In [9]:
import ds.utils.db.DB
import ds.utils.db.executeBatch

DB.executeBatch("""
	INSERT INTO airlines.forum_profiles (id, nickname, sex, first_name, last_name)
	VALUES (?, ?, ?, ?, ?);
""", data.profiles) { profile ->
    setObject(1, profile.id)
    setString(2, profile.nickname)
    setString(3, profile.sex)
    setString(4, profile.realName.firstName)
    setString(5, profile.realName.lastName)
    addBatch()
}

In [10]:
DB.executeBatch("""
	INSERT INTO airlines.loyality_program (profile_id, status, program, type)
	VALUES (?, ?, ?, ?);
""", data.profiles) { profile ->
    profile.loyalityProgram.forEach { loyalityProgram ->
        setObject(1, profile.id)
        setString(2, loyalityProgram.status)
        setString(3, loyalityProgram.program)
        setString(4, loyalityProgram.type)
        addBatch()
    }
}

In [11]:
import ds.types.raw.forum.Airport

val airports = HashMap<String, Airport>()

data.profiles.forEach { profile ->
    profile.registeredFlights.forEach { flight ->
        airports[flight.departure.airport] = flight.departure
        airports[flight.arrival.airport] = flight.arrival
    }
}

DB.executeBatch("""
	INSERT INTO airlines.airport (code, city, country)
	VALUES (?, ?, ?) ON CONFLICT DO NOTHING;
""", airports.values.toList()) { airport ->
	setString(1, airport.airport)
	setString(2, airport.city)
	setString(3, airport.country)
	addBatch()
}

In [12]:
import java.sql.Date
import java.time.format.DateTimeFormatter

val dateFormatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

DB.executeBatch("""
	INSERT INTO airlines.forum_profiles_flights (profile_id, date, depature, arrival)
	VALUES (?, ?, ?, ?);
""", data.profiles) { profile ->
    profile.registeredFlights.forEach { flight ->
        setObject(1, profile.id)
        setDate(2, Date.valueOf(java.time.LocalDate.parse(flight.date, dateFormatter)))
        setString(3, flight.arrival.airport)
        setString(4, flight.departure.airport)
        addBatch()
    }
}